In [79]:
import pyart
import numpy as np
import nexradaws
import tempfile
import xarray as xr
import time
import warnings
from netCDF4 import Dataset
import datetime

warnings.filterwarnings("ignore")

In [80]:
# Central lat/lon of ARM SGP site 
site_latlon = [36.607322, -97.487643]

In [81]:
# Create temp file, connect to NEXRAD AWS, request files by date & station ID
templocation = tempfile.mkdtemp()
conn = nexradaws.NexradAwsInterface()
scans = conn.get_avail_scans(2019, 8, 22, 'KVNX') #scans = conn.get_avail_scans(2019, 11, 10, 'KVNX')

In [86]:
# Download selected volume scans
num_scans = scans[100:102]
localfiles = conn.download(num_scans, templocation)

Downloaded KVNX20190822_084514_V06
Downloaded KVNX20190822_085122_V06
2 out of 2 files downloaded...0 errors


In [87]:
sweep = np.arange(0,18)

gate_lat_site = np.nan*np.ma.ones(18)
gate_lon_site = np.nan*np.ma.ones(18)
gate_alt_site = np.nan*np.ma.ones(18)
Z_site = np.nan*np.ma.ones(18)
scan_times = []

x = 0 
for scan in localfiles.iter_success():
    radar = pyart.io.read(localfiles.success[x].filepath)
    volume_scan_t = scan.scan_time
    scan_times.append(volume_scan_t)

    # Loop through the sweeps
    for i in range(radar.nsweeps):
        slice_start, slice_end = radar.get_start_end(i)
        gate_longitude = radar.gate_longitude['data'][slice_start:slice_end,:]
        gate_latitude = radar.gate_latitude['data'][slice_start:slice_end,:]
        gate_altitude = radar.gate_altitude['data'][slice_start:slice_end,:]

        dist = np.sqrt(((gate_latitude - site_latlon[0])**2) + ((gate_longitude - site_latlon[1])**2))
        index = np.where(dist == dist.min())

        gate_lat_site[i] = gate_latitude[index[0][0], index[1][0]]
        gate_lon_site[i] = gate_longitude[index[0][0], index[1][0]]
        gate_alt_site[i] = gate_altitude[index[0][0], index[1][0]]
        Z_site[i] = radar.fields['reflectivity']['data'][slice_start+index[0][0], index[1][0]]

        #del radar

    print(volume_scan_t)
    print(Z_site)

    np.stack([gate_lat_site, gate_lon_site, gate_alt_site, Z_site])
    x +=1
    del radar

print(scan_times)


2019-08-22 08:45:14+00:00
[37.0 34.0 40.5 42.0 44.5 43.0 40.0 43.5 48.0 36.5 29.5 33.5 23.5 18.5 2.5
 -- -- --]
2019-08-22 08:51:22+00:00
[44.5 44.5 39.5 36.0 26.5 31.0 39.0 36.5 42.5 36.5 28.0 28.5 27.0 22.5 7.0
 -- -- --]
[datetime.datetime(2019, 8, 22, 8, 45, 14, tzinfo=<UTC>), datetime.datetime(2019, 8, 22, 8, 51, 22, tzinfo=<UTC>)]


In [88]:
ds = xr.Dataset({'Z_site': (['sweep'], Z_site),
            'gate_lat_site': (['sweep'], gate_lat_site),
            'gate_lon_site': (['sweep'], gate_lon_site),
            'gate_height': (['sweep'], gate_alt_site),
            },
            coords={'time': scan_times,'sweep': sweep})

ds

<xarray.Dataset>
Dimensions:        (sweep: 18, time: 2)
Coordinates:
  * time           (time) object 2019-08-22T08:45:14+00:00 2019-08-22T08:51:2...
  * sweep          (sweep) int32 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17
Data variables:
    Z_site         (sweep) float64 44.5 44.5 39.5 36.0 26.5 ... 7.0 nan nan nan
    gate_lat_site  (sweep) float64 36.61 36.61 36.61 36.61 ... 36.61 36.61 36.61
    gate_lon_site  (sweep) float64 -97.49 -97.49 -97.49 ... -97.49 -97.49 -97.49
    gate_height    (sweep) float64 1.083e+03 1.083e+03 ... 1.825e+04 2.15e+04

In [89]:
ds = xr.Dataset({'Z_site': (['time','sweep'], Z_site),
            'gate_lat_site': (['time','sweep'], gate_lat_site),
            'gate_lon_site': (['time','sweep'], gate_lon_site),
            'gate_height': (['time','sweep'], gate_alt_site),
            },
            coords={'time': scan_times,'sweep': sweep})

ds

ValueError: Could not convert tuple of form (dims, data[, attrs, encoding]): (['time', 'sweep'], masked_array(data=[44.5, 44.5, 39.5, 36.0, 26.5, 31.0, 39.0, 36.5, 42.5,
                   36.5, 28.0, 28.5, 27.0, 22.5, 7.0, --, --, --],
             mask=[False, False, False, False, False, False, False, False,
                   False, False, False, False, False, False, False,  True,
                    True,  True],
       fill_value=1e+20)) to Variable.